## Integrantes:

- Kevin Pareja (20196318)

- Elian Tongombol (20196453)

- Paola Aranda (20196052)

- María Alejandra Colán (20190515)

In [2]:
# Importamos las librerias necesarias
import pandas as pd 
import numpy as np
import re 
from tqdm import tqdm  # controlar el tiempo en un loop
import os

# Librerias correspondientes al modelo de regresión lineal 
import statsmodels.api as sm  # linear regression utiliza todas las columnas de base de datos 
import statsmodels.formula.api as smf  # linear regression usa uan formula
from sklearn import datasets, linear_model # models 
from sklearn.metrics import mean_squared_error, r2_score
from linearmodels.iv import IV2SLS # for IV regression

import warnings
warnings.filterwarnings('ignore') # eliminar warning messages 

# Incluimos la libreriia que nos servía para exportar las tablas a Latex
from pystout import pystout

In [13]:
# Cargando la base de datos correspondiente 
datos = pd.read_stata(r"../../data/trabajo_final/replicacion/SerraPorterAEJ.dta",
                           convert_categoricals=False)
datos

,anonid,took_fall,took_year,tookanother,numeconclass,econmajor,Major,Major_STEM,Major_Business,Major_Finance,...,ecojobs_b,ecojobs_c,ecojobs_d,ecojobs_e,ecojobs_f,ecojobs_g,ecojobs_h,major_current_a,major_current_b,survey_econmajor
0,4.0,0.0,0.0,0.0,0.0,0.0,Mathematics - BS,1.0,0.0,0.0,...,Entrepreneur,Journalist/ Correspondent,Working for marketplace on NPR,Business consultant,,,,Mathematics,,0.0
1,5.0,0.0,0.0,0.0,0.0,0.0,Biological Sciences - BS,1.0,0.0,0.0,...,Tax advisor,Managerial Position,,,,,,Biology,,0.0
2,7.0,1.0,1.0,1.0,13.0,1.0,Economics w/ Fina Appl - BS,0.0,0.0,0.0,...,,,,,,,,,,NaN
3,9.0,0.0,0.0,0.0,0.0,0.0,Journalism - BA,0.0,0.0,0.0,...,,,,,,,,Journalism,Communications,0.0
4,10.0,0.0,0.0,1.0,1.0,0.0,Markets & Culture - BA,0.0,0.0,0.0,...,Investment Banker,,,,,,,Markets and Culture,Economics,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,1386.0,0.0,0.0,0.0,0.0,0.0,Markets & Culture - BA,0.0,0.0,0.0,...,,,,,,,,,,NaN
623,1388.0,0.0,0.0,0.0,0.0,0.0,Economics w/ Fina Appl - PMJ,0.0,0.0,0.0,...,,,,,,,,Business,,0.0
624,1389.0,0.0,0.0,0.0,0.0,0.0,Accounting - BBA,0.0,1.0,0.0,...,Finance,Bank,Government,,,,,Economics,Public Relations,1.0
625,1393.0,0.0,1.0,1.0,10.0,1.0,Economics - BA,0.0,0.0,0.0,...,Professor,Economist,,,,,,Entrepreneurship,Finance Economics,0.0


In [12]:
# Incluimos el resumen de los estadísticos
datos.describe()

# Analizando el tipo de variables
datos.info() # A partir de esta información se escogerá la información solicitada

datos.dtypes


<class 'pandas.core.frame.DataFrame'>
Int64Index: 627 entries, 0 to 626
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   anonid            627 non-null    float32
 1   took_fall         627 non-null    float32
 2   took_year         627 non-null    float32
 3   tookanother       627 non-null    float32
 4   numeconclass      627 non-null    float32
 5   econmajor         627 non-null    float32
 6   Major             627 non-null    object 
 7   Major_STEM        627 non-null    float32
 8   Major_Business    627 non-null    float32
 9   Major_Finance     627 non-null    float32
 10  Major_Marketing   627 non-null    float32
 11  Major_SocSc       627 non-null    float32
 12  Major_Arts        627 non-null    float32
 13  Major_Comm        627 non-null    float32
 14  Major_Hum         627 non-null    float32
 15  female            627 non-null    float32
 16  greek             501 non-null    float64
 1

anonid              float32
took_fall           float32
took_year           float32
tookanother         float32
numeconclass        float32
econmajor           float32
Major                object
Major_STEM          float32
Major_Business      float32
Major_Finance       float32
Major_Marketing     float32
Major_SocSc         float32
Major_Arts          float32
Major_Comm          float32
Major_Hum           float32
female              float32
greek               float64
econ_hs             float32
varsity             float64
class_fe            float32
class_fe2           float32
yr_2016             float32
treatment_class     float32
treat2016           float32
female_prof         float32
instate             float32
freshman            float32
american            float32
ACumGPA             float64
gradePrinciples     float32
small_class         float32
grade3301           float32
fem2016             float32
femtreat16          float32
female_treat        float32
Testscore3301       

###  1. Estadisticas descriptivas

In [15]:
# Seleccionamos las variables para las estadísticas descriptivas
table1 = data.loc[:,["american", "instate", "freshman","ACumGPA", "greek", "econ_hs", "varsity"]]
table1 

,american,instate,freshman,ACumGPA,greek,econ_hs,varsity
0,1.0,1.0,0.0,3.267,0.0,1.0,0.0
1,1.0,1.0,0.0,3.358,0.0,1.0,0.0
2,1.0,0.0,1.0,1.775,NaN,NaN,NaN
3,1.0,1.0,0.0,3.654,1.0,1.0,0.0
4,1.0,1.0,0.0,3.000,1.0,1.0,0.0
...,...,...,...,...,...,...,...
622,1.0,0.0,1.0,2.653,NaN,NaN,NaN
623,0.0,0.0,1.0,3.200,0.0,0.0,0.0
624,0.0,0.0,1.0,3.677,0.0,0.0,1.0
625,0.0,0.0,1.0,3.500,0.0,0.0,0.0


In [16]:
# Realizamos la descripción de la información previamente definida
table1.describe()

,american,instate,freshman,ACumGPA,greek,econ_hs,varsity
count,627.000000,627.000000,627.000000,627.000000,501.000000,501.000000,500.000000
mean,0.902711,0.221691,0.864434,3.432091,0.642715,0.578842,0.072000
std,0.296587,0.415717,0.342602,0.435175,0.479679,0.494239,0.258747
min,0.000000,0.000000,0.000000,1.200000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,1.000000,3.229000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,3.500000,1.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,3.740000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000


In [18]:
# Realizamos una tabla resumen que contenga información sobre:  (mean, sd, min, max, count) 
# posteriormene transponemos la tabla
summary_table = table1.describe().loc[["mean","std","count","min","max"]].T
summary_table

,mean,std,count,min,max
american,0.902711,0.296587,627.0,0.0,1.0
instate,0.221691,0.415717,627.0,0.0,1.0
freshman,0.864434,0.342602,627.0,0.0,1.0
ACumGPA,3.432091,0.435175,627.0,1.2,4.0
greek,0.642715,0.479679,501.0,0.0,1.0
econ_hs,0.578842,0.494239,501.0,0.0,1.0
varsity,0.072000,0.258747,500.0,0.0,1.0


In [19]:
# Consideramos el nombre de cada colmuna para su posterior modificación
table1.columns

Index(['american', 'instate', 'freshman', 'ACumGPA', 'greek', 'econ_hs',
       'varsity'],
      dtype='object')

In [20]:
# Cambiando los nombres
table1.columns

new_names = ["Estudiante americano",
             "Estudiante in-state",
             "Estudiante es de primer año",
             "GPA acumulado",
             "Estudiante pertenece a una fraternidad o sororidad",
             "Estudiante llevó Economía en High School",
             "Estudiante es un atleta"]
# Realizamos la unión de la estrcutra bajo un diccionario
dict( zip( table1.columns, new_names) )

{'american': 'Estudiante americano',
 'instate': 'Estudiante in-state',
 'freshman': 'Estudiante es de primer año',
 'ACumGPA': 'GPA acumulado',
 'greek': 'Estudiante pertenece a una fraternidad o sororidad',
 'econ_hs': 'Estudiante llevó Economía en High School',
 'varsity': 'Estudiante es un atleta'}

In [22]:
# Optimizando la tabla de estadísticas descriptivas
nuevos_nombres = dict( zip( table1.columns, new_names) )

columns_nuevos_nombres = {
    "mean": "Mean",
    "std": "Standard Deviation",
    "count": "Observations",
    "min": "Min",
    "max": "Max"}

# Ahora la tabla tiene nuevos nombres tando de columnas como de filas
summary_table.rename(index=nuevos_nombres, columns=columns_nuevos_nombres, inplace=True)
summary_table

,Mean,Standard Deviation,Observations,Min,Max
Estudiante americano,0.902711,0.296587,627.0,0.0,1.0
Estudiante in-state,0.221691,0.415717,627.0,0.0,1.0
Estudiante es de primer año,0.864434,0.342602,627.0,0.0,1.0
GPA acumulado,3.432091,0.435175,627.0,1.2,4.0
Estudiante pertenece a una fraternidad o sororidad,0.642715,0.479679,501.0,0.0,1.0
Estudiante llevó Economía en High School,0.578842,0.494239,501.0,0.0,1.0
Estudiante es un atleta,0.072000,0.258747,500.0,0.0,1.0


In [24]:
# Exportamos a Latex
summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2)\
.format(subset="Observations", precision=0).format(subset="Min", precision=0).format(subset="Max", precision=0)\
.to_latex(
    r"..\..\trabajo_final\Grupo11\Tabla_ED.tex",
caption="Descriptive Statistics",
    column_format = "lccc"   # l: left, c:center , r: right
)

### 2. Replica de tablas

#### 2.1 Tabla 3

#### 2.3 Tabla 5

#### 2.4 Tabla 6

### 3. Modelo econometricos


#### 3.1 Tabla 3

#### 3.2 Tabla 4

#### 3.3 Tabla 5

#### 3.4 Tabla 6

###  4. Creación de un coefplot de la tabla 4